In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ventilator-pressure-prediction/sample_submission.csv
/kaggle/input/ventilator-pressure-prediction/train.csv
/kaggle/input/ventilator-pressure-prediction/test.csv


In [2]:
##Analyze the ventillator pressure dataset
import pandas as pd
import numpy as np
##Create a NN model
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM
from tensorflow.keras.callbacks import CSVLogger, TensorBoard, ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.preprocessing import timeseries_dataset_from_array

DEBUG = False

trainDataPath = "/kaggle/input/ventilator-pressure-prediction/train.csv"
df_data = pd.read_csv(trainDataPath, header=0)

df_data.head()
#df_data.describe()

testDataPath = "/kaggle/input/ventilator-pressure-prediction/test.csv"
df_test = pd.read_csv(testDataPath, header=0)

print(df_test.head())

if DEBUG:
    df_data = df_data.iloc[:80*100, :]
    df_test = df_test.iloc[:80*100, :]
print(df_data.shape)
print(df_test.shape)

   id  breath_id  R   C  time_step       u_in  u_out
0   1          0  5  20   0.000000   0.000000      0
1   2          0  5  20   0.031904   7.515046      0
2   3          0  5  20   0.063827  14.651675      0
3   4          0  5  20   0.095751  21.230610      0
4   5          0  5  20   0.127644  26.320956      0
(6036000, 8)
(4024000, 7)


In [3]:
# df_data = df_data.iloc[0:240, :]
# df_data["u_in_prev1"] = df_data.groupby("breath_id")["u_in"].shift(1, fill_value=0)
# df_data["u_in_prev2"] = df_data.groupby("breath_id")["u_in"].shift(2, fill_value=0)
# df_data["u_in_prev3"] = df_data.groupby("breath_id")["u_in"].shift(3, fill_value=0)
# df_data["u_in_next1"] = df_data.groupby("breath_id")["u_in"].shift(-1, fill_value=0)
# df_data["u_in_next2"] = df_data.groupby("breath_id")["u_in"].shift(-2, fill_value=0)
# df_data["u_in_next3"] = df_data.groupby("breath_id")["u_in"].shift(-3, fill_value=0)
# df_data["u_in_cumm"] = df_data.groupby("breath_id")["u_in"].cumsum()
# df_data["area"] = df_data["time_step"]*df_data["u_in"]
# df_data["area"] = df_data.groupby("breath_id")["area"].cumsum()
# df_data["u_in_diff1"] = df_data["u_in"] - df_data["u_in_prev1"]
# df_data["u_in_diff2"] = df_data["u_in"] - df_data["u_in_prev2"]
# df_data["u_in_diff3"] = df_data["u_in"] - df_data["u_in_prev3"]

# df_data["u_out_prev1"] = df_data.groupby("breath_id")["u_out"].shift(1, fill_value=0)
# df_data["u_out_prev2"] = df_data.groupby("breath_id")["u_out"].shift(2, fill_value=0)
# df_data["u_out_prev3"] = df_data.groupby("breath_id")["u_out"].shift(3, fill_value=0)
# df_data["u_out_next1"] = df_data.groupby("breath_id")["u_out"].shift(-1, fill_value=0)
# df_data["u_out_next2"] = df_data.groupby("breath_id")["u_out"].shift(-2, fill_value=0)
# df_data["u_out_next3"] = df_data.groupby("breath_id")["u_out"].shift(-3, fill_value=0)
# df_data["u_out_diff1"] = df_data["u_out"] - df_data["u_out_prev1"]
# df_data["u_out_diff2"] = df_data["u_out"] - df_data["u_out_prev2"]
# df_data["u_out_diff3"] = df_data["u_out"] - df_data["u_out_prev3"]

# df_data["u_in_diff_max"] = df_data.groupby("breath_id")["u_in"].transform("max") - df_data["u_in"]
# df_data["u_in_diff_mean"] = df_data.groupby("breath_id")["u_in"].transform("mean") - df_data["u_in"]

# df_data["u_in_u_out"] = df_data["u_in"] * df_data["u_out"]
# df_data["u_out_timestep"] = df_data["u_out"] * df_data["time_step"]

# df_data['R'] = df_data['R'].astype(str)
# df_data['C'] = df_data['C'].astype(str)
# df_data['R__C'] = df_data["R"].astype(str) + '__' + df_data["C"].astype(str)
# df_data = pd.get_dummies(df_data)

# print(df_data.head())
# print(df_data.iloc[75:85,:])


In [4]:
def addFeatures(df_data):    
    #df_data = df_data.iloc[0:240, :]
    df_data["u_in_prev1"] = df_data.groupby("breath_id")["u_in"].shift(1, fill_value=0)
    df_data["u_in_prev2"] = df_data.groupby("breath_id")["u_in"].shift(2, fill_value=0)
    df_data["u_in_prev3"] = df_data.groupby("breath_id")["u_in"].shift(3, fill_value=0)
    df_data["u_in_next1"] = df_data.groupby("breath_id")["u_in"].shift(-1, fill_value=0)
    df_data["u_in_next2"] = df_data.groupby("breath_id")["u_in"].shift(-2, fill_value=0)
    df_data["u_in_next3"] = df_data.groupby("breath_id")["u_in"].shift(-3, fill_value=0)
    df_data["u_in_cumm"] = df_data.groupby("breath_id")["u_in"].cumsum()
    df_data["area"] = df_data["time_step"]*df_data["u_in"]
    df_data["area"] = df_data.groupby("breath_id")["area"].cumsum()
    df_data["u_in_diff1"] = df_data["u_in"] - df_data["u_in_prev1"]
    df_data["u_in_diff2"] = df_data["u_in"] - df_data["u_in_prev2"]
    df_data["u_in_diff3"] = df_data["u_in"] - df_data["u_in_prev3"]

    df_data["u_out_prev1"] = df_data.groupby("breath_id")["u_out"].shift(1, fill_value=0)
    df_data["u_out_prev2"] = df_data.groupby("breath_id")["u_out"].shift(2, fill_value=0)
    df_data["u_out_prev3"] = df_data.groupby("breath_id")["u_out"].shift(3, fill_value=0)
    df_data["u_out_next1"] = df_data.groupby("breath_id")["u_out"].shift(-1, fill_value=0)
    df_data["u_out_next2"] = df_data.groupby("breath_id")["u_out"].shift(-2, fill_value=0)
    df_data["u_out_next3"] = df_data.groupby("breath_id")["u_out"].shift(-3, fill_value=0)
    df_data["u_out_diff1"] = df_data["u_out"] - df_data["u_out_prev1"]
    df_data["u_out_diff2"] = df_data["u_out"] - df_data["u_out_prev2"]
    df_data["u_out_diff3"] = df_data["u_out"] - df_data["u_out_prev3"]

    df_data["u_in_diff_max"] = df_data.groupby("breath_id")["u_in"].transform("max") - df_data["u_in"]
    df_data["u_in_diff_mean"] = df_data.groupby("breath_id")["u_in"].transform("mean") - df_data["u_in"]

    df_data["u_in_u_out"] = df_data["u_in"] * df_data["u_out"]
    df_data["u_out_timestep"] = df_data["u_out"] * df_data["time_step"]

    df_data['R'] = df_data['R'].astype(str)
    df_data['C'] = df_data['C'].astype(str)
    df_data['R__C'] = df_data["R"].astype(str) + '__' + df_data["C"].astype(str)
    df_data = pd.get_dummies(df_data)
    
    return df_data

In [5]:
##Added features for data and test
df_data = addFeatures(df_data)
df_test = addFeatures(df_test)
print(df_data.shape)
print(df_test.shape)

(6036000, 45)
(4024000, 44)


In [6]:
##Shuffle the data and get the features similar for train and test
df_data = df_data.sample(frac=1)
df_y_data = df_data["pressure"].copy()
df_data = df_data.drop(["id", "breath_id", "pressure"], axis=1)

df_test = df_test.drop(["id", "breath_id"], axis=1)
print("data shape= %s, Test shape= %s" %(df_data.shape, df_test.shape))

data shape= (6036000, 42), Test shape= (4024000, 42)


In [7]:
##Normalize the data
from sklearn.preprocessing import RobustScaler, normalize
RS = RobustScaler()
train = RS.fit_transform(df_data)
test = RS.transform(df_test)
print(train[0:5,:])
print(test[0:5,:])

[[ 8.49539496e-01  1.23978908e-01  0.00000000e+00  1.30897170e-01
   1.49604370e-01  1.71568393e-01  1.27060415e-01  1.39034497e-01
   1.56505118e-01 -1.71809269e-01  7.25702978e-01  3.73311057e-03
   5.74646500e-03  5.38913779e-03  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -4.37269938e-01
  -5.42666078e-01  1.10743958e+00  5.71662094e-01  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-9.12645320e-01  1.19168704e+01 -1.00000000e+00  1.75780689e+01
   1.92482221e+01  7.77138501e+00  8.44291715e+00  5.81602126e+00
   3.55079007e+00  4.72898017e-02 -4.52659547e-01 -2.02316592e+02
  -1.30936925e+02  3.39013914e+01 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00 -1.0000

In [8]:
##Shuffle df_data and then get df_y_train and df_x_train
from sklearn.model_selection import train_test_split

## Remove random_state if we want variable train and dev sizes
#x_train, x_dev = train_test_split(train, train_size=0.8, random_state=9)
x_train, x_dev, y_train, y_dev = train_test_split(df_data, df_y_data, train_size=0.8)

print("data shape= %s, train size= %s, dev size = %s\n" %(df_data.shape, x_train.shape, x_dev.shape))

data shape= (6036000, 42), train size= (4828800, 42), dev size = (1207200, 42)



In [9]:
##Create FNN for training
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.callbacks import CSVLogger, TensorBoard, ModelCheckpoint, EarlyStopping

##Clear models
tf.keras.backend.clear_session()

model = Sequential()
model.add(tf.keras.Input(shape=(42,)))
model.add(Dense(256,activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               11008     
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_3 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 438,145
Trainable params: 438,145
Non-trainable params: 0
_________________________________________________________________



User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_P

In [10]:
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True,\
                                             verbose=1)

##Fit model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.MeanAbsoluteError()])

##Train the model
history = model.fit(x=x_train, y=y_train, batch_size=1024, epochs=100, validation_data=(x_dev, y_dev),\
                   callbacks=[earlystop])

2021-10-18 14:59:53.880803: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
4716/4716 [==============================] - 167s 35ms/step - loss: 8.2080 - mean_absolute_error: 1.3267 - val_loss: 1.9629 - val_mean_absolute_error: 0.8676
Epoch 2/100
4716/4716 [==============================] - 167s 35ms/step - loss: 1.6786 - mean_absolute_error: 0.7975 - val_loss: 1.2881 - val_mean_absolute_error: 0.6884
Epoch 3/100
4716/4716 [==============================] - 168s 36ms/step - loss: 1.3062 - mean_absolute_error: 0.6990 - val_loss: 1.1836 - val_mean_absolute_error: 0.6535
Epoch 4/100
4716/4716 [==============================] - 168s 36ms/step - loss: 1.1069 - mean_absolute_error: 0.6405 - val_loss: 0.9818 - val_mean_absolute_error: 0.5991
Epoch 5/100
4716/4716 [==============================] - 169s 36ms/step - loss: 0.9916 - mean_absolute_error: 0.6042 - val_loss: 1.0375 - val_mean_absolute_error: 0.6099
Epoch 6/100
4716/4716 [==============================] - 168s 36ms/step - loss: 0.9175 - mean_absolute_error: 0.5805 - val_loss: 0.8788 - val_mean_abs

In [11]:
model.save("./ventilator_extraFeatures_fnn1.h5")
##Evaluate model on dev set
score = model.evaluate(x_dev, y_dev, batch_size=10240)
score

118/118 [==============================] - 10s 81ms/step - loss: 0.4587 - mean_absolute_error: 0.4147


[0.45874926447868347, 0.41471993923187256]

In [12]:
df_sub = pd.read_csv("/kaggle/input/ventilator-pressure-prediction/sample_submission.csv", header=0)
df_sub["pressure"] = model.predict(df_test, batch_size=10240)
df_sub.to_csv("./ventilator_extraFeatures_fnn1.csv", index=False)
print("Done saving file")

Done saving file
